# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

 Could not import win32api and/or win32con


2025-02-03 14:42:26.412965: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 14:42:26.421972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738590146.433000   71074 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738590146.436130   71074 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 14:42:26.448178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# set the parameters to run the script
directories = [r'/media/elea/SSD Éléa/Test_processing']
calib_directory = r'/media/elea/SSD Éléa/calib'
    
# set the parameters to be used for the line visualisation with the parameter parameter_set
parameter_set = 'TheoniPics'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = False

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes with PDDN when available and also without using the PDDN
PDDN_mode = 'both'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550]

# Processing mode
# 1. 'full': processes everything (azimuth_noise, all plots and visualization) - 1x speed
# 2. 'no_visualization': processes azimuth noise + all plots but no visualization - 1.25x faster
# 3. 'fast': processes only the legacy plots (i.e. depolarization, azimuth, retardance and diattenuation) - 3x faster
processing_mode = 'no_visualization'

# Wether or not time mode (to get the processing times is activated or not)
time_mode = True

In [9]:
times, time_complete = batch_processing.batch_process_master(directories, calib_directory, run_all = run_all, 
                                parameter_set = parameter_set, PDDN = PDDN_mode, 
                                wavelengths = wavelengths, processing_mode = processing_mode,
                                time_mode = time_mode)
times

1. processing without PDDN...


  0%|                                                     | 0/1 [00:00<?, ?it/s]

['/media/elea/SSD Éléa/Test_processing/2024-09-26_T_CHB-2_FX_X1_1']
./temp_processing/2024-09-26_T_CHB-2_FX_X1_1/polarimetry/550nm ['Depolarization.png', 'Azimuth of optical axis.png', 'Intensity.png', 'Linear retardance.png']


100%|█████████████████████████████████████████████| 1/1 [00:05<00:00,  5.84s/it]


processing without PDDN done.

2. processing with PDDN.


0it [00:00, ?it/s]

processing with PDDN done.


{'MM_processing': 0.3455212116241455,
 'azimuth_std_processing': 0.68141770362854,
 'denoising': 0,
 'save_npz': 0.10818719863891602,
 'full_processing': 1.1579973697662354,
 'plotting': 4.426774978637695}

In [18]:
PDDN_mode = 'both'
run_all = False

In [22]:
from processingmm import align_wavelengths
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all, imgj_processing = False)

['Z:\\DatabaseSeg\\DB_upToDate\\batch2\\71\\2022-12-07_T_HORAO-71-BF_FR_S3_1', 'Z:\\DatabaseSeg\\DB_upToDate\\HT\\2024-09-18_T_AUTOPSY-BF_FR_S5_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch2\\71\\2023-06-08_T_HORAO-71-AF_FR_S3_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch3\\86\\2023-02-21_T_HORAO-86-BF_FR_S1_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch5\\100\\2023-05-04_T_HORAO-100-BF_FR_S1_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch5\\98\\2023-10-19_T_HORAO-98-AF_FR_S3_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch1\\67\\2022-11-22_T_HORAO-67-BF_FR_S1_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch4\\95\\2023-04-18_T_HORAO-95-BF_FR_S1_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch3\\88\\2023-03-09_T_HORAO-88-BF_FR_S1_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch2\\76\\2023-03-07_T_HORAO-76-BF_FR_S1_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch2\\76\\2023-11-02_T_HORAO-76-AF_FR_S1_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch3\\91\\2023-03-22_T_HORAO-91-BF_FR_S2_1', 'Z:\\DatabaseSeg\\DB_upToDate\\batch4\\92\\2023-11-01_T_HORAO-92-AF_FR

  0%|                                                                                          | 0/158 [00:00<?, ?it/s]

['Z:\\DatabaseSeg\\DB_upToDate\\batch2\\71\\2022-12-07_T_HORAO-71-BF_FR_S3_1\\raw_data\\600nm\\600_Intensite_PDDN_aligned.cod', 'Z:\\DatabaseSeg\\DB_upToDate\\batch2\\71\\2022-12-07_T_HORAO-71-BF_FR_S3_1\\raw_data\\600nm\\600_Intensite_aligned.cod']
['Z:\\DatabaseSeg\\DB_upToDate\\HT\\2024-09-18_T_AUTOPSY-BF_FR_S5_1\\raw_data\\600nm\\600_Intensite_PDDN_aligned.cod', 'Z:\\DatabaseSeg\\DB_upToDate\\HT\\2024-09-18_T_AUTOPSY-BF_FR_S5_1\\raw_data\\600nm\\600_Intensite_aligned.cod']
['Z:\\DatabaseSeg\\DB_upToDate\\batch2\\71\\2023-06-08_T_HORAO-71-AF_FR_S3_1\\raw_data\\600nm\\600_Intensite_PDDN_aligned.cod', 'Z:\\DatabaseSeg\\DB_upToDate\\batch2\\71\\2023-06-08_T_HORAO-71-AF_FR_S3_1\\raw_data\\600nm\\600_Intensite_aligned.cod']
['Z:\\DatabaseSeg\\DB_upToDate\\batch3\\86\\2023-02-21_T_HORAO-86-BF_FR_S1_1\\raw_data\\600nm\\600_Intensite_PDDN_aligned.cod', 'Z:\\DatabaseSeg\\DB_upToDate\\batch3\\86\\2023-02-21_T_HORAO-86-BF_FR_S1_1\\raw_data\\600nm\\600_Intensite_aligned.cod']
['Z:\\DatabaseSeg\

  9%|███████▋                                                                         | 15/158 [00:30<04:51,  2.04s/it]

['Z:\\DatabaseSeg\\DB_upToDate\\batch3\\91\\2023-03-22_T_HORAO-91-BF_FR_S1_1\\raw_data\\600nm\\600_Intensite_PDDN_aligned.cod', 'Z:\\DatabaseSeg\\DB_upToDate\\batch3\\91\\2023-03-22_T_HORAO-91-BF_FR_S1_1\\raw_data\\600nm\\600_Intensite_aligned.cod']
 
 >> Exported X3D .cod file as: Z:\DatabaseSeg\DB_upToDate\batch3\91\2023-03-22_T_HORAO-91-BF_FR_S1_1\raw_data\600nm\600_Intensite_PDDN_aligned.cod
 
 >> Exported X3D .cod file as: Z:\DatabaseSeg\DB_upToDate\batch3\91\2023-03-22_T_HORAO-91-BF_FR_S1_1\raw_data\600nm\600_Intensite_aligned.cod


...

 99%|██████████████████████████████████████████████████████████████████████████████▉ | 156/158 [46:38<00:44, 22.10s/it]

['Z:\\DatabaseSeg\\DB_upToDate\\batch5\\106\\2023-07-25_T_HORAO-106-BF_FR_S1_1\\raw_data\\600nm\\600_Intensite_PDDN_aligned.cod', 'Z:\\DatabaseSeg\\DB_upToDate\\batch5\\106\\2023-07-25_T_HORAO-106-BF_FR_S1_1\\raw_data\\600nm\\600_Intensite_aligned.cod']
 
 >> Exported X3D .cod file as: Z:\DatabaseSeg\DB_upToDate\batch5\106\2023-07-25_T_HORAO-106-BF_FR_S1_1\raw_data\600nm\600_Intensite_PDDN_aligned.cod
 
 >> Exported X3D .cod file as: Z:\DatabaseSeg\DB_upToDate\batch5\106\2023-07-25_T_HORAO-106-BF_FR_S1_1\raw_data\600nm\600_Intensite_aligned.cod


 99%|███████████████████████████████████████████████████████████████████████████████▍| 157/158 [47:03<00:22, 22.98s/it]

['Z:\\DatabaseSeg\\DB_upToDate\\batch5\\100\\2023-06-07_T_HORAO-100-AF_FR_S2_1\\raw_data\\600nm\\600_Intensite_PDDN_aligned.cod', 'Z:\\DatabaseSeg\\DB_upToDate\\batch5\\100\\2023-06-07_T_HORAO-100-AF_FR_S2_1\\raw_data\\600nm\\600_Intensite_aligned.cod']
 
 >> Exported X3D .cod file as: Z:\DatabaseSeg\DB_upToDate\batch5\100\2023-06-07_T_HORAO-100-AF_FR_S2_1\raw_data\600nm\600_Intensite_PDDN_aligned.cod
 
 >> Exported X3D .cod file as: Z:\DatabaseSeg\DB_upToDate\batch5\100\2023-06-07_T_HORAO-100-AF_FR_S2_1\raw_data\600nm\600_Intensite_aligned.cod


100%|████████████████████████████████████████████████████████████████████████████████| 158/158 [47:28<00:00, 18.03s/it]


In [ ]:
from processingmm import add_calibration
add_calibration.add_calibration(directories)